In [1]:
%matplotlib inline
from tensorflow.keras import applications
from tensorflow.keras.datasets import mnist
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Convolution2D, MaxPooling2D
from keras.utils import np_utils
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import train_test_split
from tensorflow.keras import optimizers

Using TensorFlow backend.


# Initial Settings

In [2]:
img_size = 100
_lambda = 0.01

# Load Data

In [3]:
data = np.load('processed_val_images.npy')
labels = np.load('processed_val_labels.npy')

print(data.shape)
print(labels.shape)

(313, 100, 100, 3)
(313,)


# Train Test Split

In [4]:
X_train, X_test, y_train, y_test = train_test_split(data, labels, train_size = 0.8, random_state = 20)

nb_classes = 2
y_train = np.array(y_train)
y_test = np.array(y_test)
Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

print("Training matrix shape", X_train.shape)
print("Testing matrix shape", X_test.shape)

print("Training label shape", Y_train.shape)
print("Testing label shape", Y_test.shape)

Training matrix shape (250, 100, 100, 3)
Testing matrix shape (63, 100, 100, 3)
Training label shape (250, 2)
Testing label shape (63, 2)


# Google's Inception V3

In [5]:
# from keras.applications.inception_v3 import InceptionV3
# from keras.applications.inception_v3 import preprocess_input
# from keras.applications.inception_v3 import decode_predictions

# model = InceptionV3(weights='imagenet', include_top=True)

# VGG16

https://github.com/fchollet/deep-learning-models

In [6]:
# model = applications.VGG16(weights = "imagenet", include_top=False, input_shape = (img_size, img_size, 3))

In [7]:
# top_model = Sequential()
# top_model.add(Flatten(input_shape=model.output_shape[1:]))
# top_model.add(Dense(256, activation='relu'))
# top_model.add(Dropout(0.5))
# top_model.add(Dense(1, activation='sigmoid'))

# # note that it is necessary to start with a fully-trained
# # classifier, including the top classifier,
# # in order to successfully do fine-tuning
# top_model.load_weights(top_model_weights_path)

# # add the model on top of the convolutional base
# model.add(top_model)

# for layer in model.layers[:25]:
#     layer.trainable = False

# # compile the model with a SGD/momentum optimizer
# # and a very slow learning rate.
# model.compile(loss='binary_crossentropy',
#               optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
#               metrics=['accuracy'])

# VGG 14

In [8]:
# model = applications.VGG14(weights = "imagenet", include_top=False, input_shape = (img_size, img_size, 3))

In [9]:
# mode

NameError: name 'mode' is not defined

# VGG-19

In [5]:
model = applications.VGG19(weights = "imagenet", include_top=False, input_shape = (img_size, img_size, 3))

In [7]:
model.summary()

Model: "vgg19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 100, 100, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 100, 100, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 100, 100, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 50, 50, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 50, 50, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 50, 50, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 25, 25, 128)       0     

In [ ]:
## Make last two layers trainable


# Create FC Layer

In [8]:
top_model = Sequential()

top_model.add(Flatten(input_shape=model.output_shape[1:]))
top_model.add(Dense(128, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(2, activation='softmax'))

# top_model.load_weights("first_try.h5")

In [9]:
new_model = Sequential() #new model
for layer in model.layers: 
    new_model.add(layer)
    
new_model.add(top_model) # now this works

In [10]:
for layer in model.layers[:21]:
    layer.trainable = False

In [11]:
adam = optimizers.SGD(lr=1e-4, decay=1e-6, momentum=0.9, nesterov=True)
new_model.compile(loss='categorical_crossentropy',
              optimizer= adam,
              metrics=['accuracy'])

In [13]:
new_model.fit(X_train, Y_train, 
          batch_size=64, epochs=10, verbose=2 )

new_model.save('model_V4.h5')

Epoch 1/10
4/4 - 1s - loss: 0.6492 - accuracy: 0.6440
Epoch 2/10
4/4 - 0s - loss: 0.5787 - accuracy: 0.7160
Epoch 3/10
4/4 - 0s - loss: 0.5331 - accuracy: 0.7680
Epoch 4/10
4/4 - 0s - loss: 0.5657 - accuracy: 0.7800
Epoch 5/10
4/4 - 0s - loss: 0.5289 - accuracy: 0.8200
Epoch 6/10
4/4 - 0s - loss: 0.5551 - accuracy: 0.8040
Epoch 7/10
4/4 - 0s - loss: 0.5047 - accuracy: 0.7920
Epoch 8/10
4/4 - 0s - loss: 0.5014 - accuracy: 0.8080
Epoch 9/10
4/4 - 0s - loss: 0.4964 - accuracy: 0.8080
Epoch 10/10
4/4 - 0s - loss: 0.5361 - accuracy: 0.7960


In [14]:
score = new_model.evaluate(X_test, Y_test, verbose=0)
print("Loss: " + str(score[0]))
print("Accuracy: " + str(score[1]))
new_model.save('model_V4.h5')

Loss: 0.4570825397968292
Accuracy: 0.8253968358039856


In [15]:
from keras.models import model_from_json
model_json = new_model.to_json()
with open("newmodel.json","w") as json_file:
    json_file.write(model_json)
model.save_weights("newmodel.h5")
print("Saved model to disk")

Saved model to disk


# Defining the Model

In [17]:
model = Sequential()
# model.add(Convolution2D(32, 3, 3, activation='relu', input_shape=(img_size, img_size, 3)))
# model.add(MaxPooling2D(pool_size=(2,2)))
# model.add(Dropout(0.25))

# model.add(Convolution2D(32, 3, 3, activation='relu', input_shape=(img_size, img_size, 3)))
# model.add(MaxPooling2D(pool_size=(2,2)))
# model.add(Dropout(0.25))

# model.add(Convolution2D(32, 3, 3, activation='relu', input_shape=(img_size, img_size, 3)))
# model.add(MaxPooling2D(pool_size=(2,2)))
# model.add(Dropout(0.25))

model.add(Flatten(input_shape=(img_size, img_size, 3)))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))

adam = optimizers.SGD(lr=1e-4, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy',
              optimizer= adam,
              metrics=['accuracy'])


model.fit(X_train, Y_train, 
          batch_size=64, epochs=10, verbose=2)

Epoch 1/10
4/4 - 0s - loss: 0.5834 - accuracy: 0.7400
Epoch 2/10
4/4 - 0s - loss: 0.5773 - accuracy: 0.7880
Epoch 3/10
4/4 - 0s - loss: 0.5538 - accuracy: 0.7880
Epoch 4/10
4/4 - 0s - loss: 0.5220 - accuracy: 0.8080
Epoch 5/10
4/4 - 0s - loss: 0.5124 - accuracy: 0.8080
Epoch 6/10
4/4 - 0s - loss: 0.5190 - accuracy: 0.8160
Epoch 7/10
4/4 - 0s - loss: 0.4865 - accuracy: 0.8120
Epoch 8/10
4/4 - 0s - loss: 0.5026 - accuracy: 0.8160
Epoch 9/10
4/4 - 0s - loss: 0.4960 - accuracy: 0.8200
Epoch 10/10
4/4 - 0s - loss: 0.4892 - accuracy: 0.8120


In [18]:
score = model.evaluate(X_test, Y_test, verbose=0)
print("Loss: " + str(score[0]))
print("Accuracy: " + str(score[1]))

Loss: 0.4693923592567444
Accuracy: 0.8253968358039856


# 3-Layer Convolution Model

In [21]:
model = Sequential()
model.add(Convolution2D(32, 3, 3, activation='relu', input_shape=(img_size, img_size, 3)))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Convolution2D(32, 3, 3, activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Convolution2D(64, 3, 3, activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
          
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))

adam = optimizers.SGD(lr=1e-4, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy',
              optimizer= adam,
              metrics=['accuracy'])


model.fit(X_train, Y_train, 
          batch_size=64, epochs=10, verbose=2)

Epoch 1/10
4/4 - 0s - loss: 0.6907 - accuracy: 0.5240
Epoch 2/10
4/4 - 0s - loss: 0.6999 - accuracy: 0.4840
Epoch 3/10
4/4 - 0s - loss: 0.6858 - accuracy: 0.5800
Epoch 4/10
4/4 - 0s - loss: 0.6819 - accuracy: 0.5520
Epoch 5/10
4/4 - 0s - loss: 0.6749 - accuracy: 0.5960
Epoch 6/10
4/4 - 0s - loss: 0.6710 - accuracy: 0.6440
Epoch 7/10
4/4 - 0s - loss: 0.6632 - accuracy: 0.6920
Epoch 8/10
4/4 - 0s - loss: 0.6574 - accuracy: 0.7040
Epoch 9/10
4/4 - 0s - loss: 0.6562 - accuracy: 0.7120
Epoch 10/10
4/4 - 0s - loss: 0.6427 - accuracy: 0.7600


In [22]:
model.predict_proba(X_test, verbose=1)

Instructions for updating:
Please use `model.predict()` instead.
2/2 [==============================] - 0s 7ms/step


array([[0.54903734, 0.45096266],
       [0.5403585 , 0.45964152],
       [0.5621946 , 0.43780535],
       [0.5594473 , 0.44055268],
       [0.5359233 , 0.46407667],
       [0.5552391 , 0.4447609 ],
       [0.54377705, 0.45622292],
       [0.5434544 , 0.4565456 ],
       [0.5357815 , 0.46421853],
       [0.5317472 , 0.46825275],
       [0.5437384 , 0.45626158],
       [0.53418106, 0.46581897],
       [0.55072314, 0.44927692],
       [0.54427475, 0.45572525],
       [0.5325518 , 0.46744823],
       [0.5573463 , 0.44265375],
       [0.5499232 , 0.45007682],
       [0.5526536 , 0.4473464 ],
       [0.55507123, 0.44492882],
       [0.550805  , 0.449195  ],
       [0.5507215 , 0.4492785 ],
       [0.54372233, 0.45627764],
       [0.5311088 , 0.4688912 ],
       [0.53596723, 0.46403277],
       [0.56718373, 0.43281624],
       [0.52855504, 0.47144493],
       [0.5504382 , 0.44956174],
       [0.5452001 , 0.45479986],
       [0.53936124, 0.46063876],
       [0.5553281 , 0.4446719 ],
       [0.

In [23]:
score = model.evaluate(X_test, Y_test, verbose=0)
print("Loss: " + str(score[0]))
print("Accuracy: " + str(score[1]))

Loss: 0.6415705680847168
Accuracy: 0.8253968358039856


In [24]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))


Num GPUs Available:  1
